In [1]:
import keras_tuner
from tensorflow import keras
from keras_tuner.tuners import RandomSearch
import pandas as pd

Using TensorFlow backend


2024-01-17 09:18:15.722898: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-17 09:18:15.922636: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 09:18:15.922682: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 09:18:15.923928: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-17 09:18:16.045876: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-17 09:18:16.047748: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [3]:
#adv_df = pd.read_csv("./data/shadow_model_outputs.csv")
adv_df = pd.read_csv("data/shadow_model_outputs_proba-new.csv")
adv_ddf_shuffled = adv_df.sample(frac=1, random_state=1).reset_index(drop=True)#.drop(columns=["Unnamed: 0"])
adv_y = adv_ddf_shuffled["y"]
adv_X = adv_ddf_shuffled.drop(columns=["y"])

In [4]:
adv_df_test = pd.read_csv("data/shadow_model_outputs_proba-new_test_set.csv")
#adv_df_test = pd.read_csv("./data/shadow_model_outputs_test_set.csv")
adv_ddf_shuffled_test = adv_df_test.sample(frac=1, random_state=1).reset_index(drop=True)#.drop(columns=["Unnamed: 0"])
adv_y_test = adv_ddf_shuffled_test["y"]
adv_X_test = adv_ddf_shuffled_test.drop(columns=["y"])

In [4]:
def build_model(hp):
    model = keras.Sequential()
    #model.add(keras.layers.Dense(units=8000, activation="relu"))
    #if hp.Boolean("lstm"):
        #model.add(keras.layers.Dropout(rate=0.1))
    #    model.add(keras.layers.Reshape((adv_X.shape[1],1)))
    #    model.add(keras.layers.LSTM(hp.Int('units_lstm', min_value=2, max_value=30, step=4), activation='relu', return_sequences=False, return_state=False))
    #else:
    #    model.add(keras.layers.Dense(units=hp.Int('units_dense1', min_value=8, max_value=64, step=8)))
    
    #model.add(keras.layers.Dense(units=hp.Int('units_dense1', min_value=45, max_value=47, step=1), activation="relu"))
    
    model.add(keras.layers.Dense(46, activation="relu"))
    
    model.add(keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.01, max_value=0.04, step=0.01)))
    
    #if hp.Boolean("dropout"):
    #model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.Dense(18, activation="relu"))
    #model.add(keras.layers.Dense(units=hp.Int('units_dense2', min_value=17, max_value=23, step=1), activation="relu"))
    model.add(keras.layers.Dense(1))
    
    model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=keras.metrics.R2Score())
    return model

In [5]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(keras.layers.Dense(hp.Int('units_dense0', min_value=38, max_value=50, step=2)))
        if hp.Boolean("relu_dense0"):
            model.add(keras.layers.Activation(keras.activations.relu))
        model.add(keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.01, max_value=0.04, step=0.01)))
        model.add(keras.layers.Dense(18))
        if hp.Boolean("relu_dense1"):
            model.add(keras.layers.Activation(keras.activations.relu))
        model.add(keras.layers.Dense(1))
        model.compile(optimizer="adam", loss=keras.losses.MeanSquaredError(), metrics=keras.metrics.R2Score())
        return model
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [40, 64, 80]),
            **kwargs,
        )


In [6]:
from keras_tuner import Objective
tuner = RandomSearch(
    MyHyperModel(),
    #objective=keras_tuner.Objective('val_r2_score', direction='max'),
    objective = Objective(name="val_r2_score", direction="max"),
    max_trials = 50,
    executions_per_trial = 4,
    tuner_id="17.01.regression.adversary.cgpt"
)

In [7]:
tuner.search(adv_X, adv_y, epochs=200, validation_data=(adv_X_test, adv_y_test), callbacks=[keras.callbacks.EarlyStopping('val_r2_score', patience=30)])

Trial 50 Complete [00h 01m 02s]
val_r2_score: 0.016619369387626648

Best val_r2_score So Far: 0.2648559659719467
Total elapsed time: 00h 40m 34s


In [8]:
models = tuner.get_best_models(num_models=5)

In [9]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_r2_score", direction="max")

Trial 45 summary
Hyperparameters:
units_dense0: 38
relu_dense0: False
dropout_rate: 0.02
relu_dense1: False
batch_size: 64
Score: 0.2648559659719467

Trial 47 summary
Hyperparameters:
units_dense0: 46
relu_dense0: False
dropout_rate: 0.03
relu_dense1: False
batch_size: 64
Score: 0.21320849657058716

Trial 26 summary
Hyperparameters:
units_dense0: 50
relu_dense0: False
dropout_rate: 0.01
relu_dense1: True
batch_size: 40
Score: 0.17600581049919128

Trial 09 summary
Hyperparameters:
units_dense0: 42
relu_dense0: False
dropout_rate: 0.02
relu_dense1: False
batch_size: 80
Score: 0.1639590561389923

Trial 01 summary
Hyperparameters:
units_dense0: 46
relu_dense0: False
dropout_rate: 0.02
relu_dense1: False
batch_size: 64
Score: 0.15083035826683044

Trial 27 summary
Hyperparameters:
units_dense0: 48
relu_dense0: True
dropout_rate: 0.03
relu_dense1: True
batch_size: 40
Score: 0.

In [35]:
best_model = models[0]

In [36]:
best_model.build(input_shape=(None,adv_X.shape[1],))

In [37]:
best_model.layers[1].rate

0.01

In [38]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 46)                920138    
                                                                 
 dropout (Dropout)           (None, 46)                0         
                                                                 
 dense_1 (Dense)             (None, 18)                846       
                                                                 
 dense_2 (Dense)             (None, 1)                 19        
                                                                 
Total params: 921003 (3.51 MB)
Trainable params: 921003 (3.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
best_model.save("adv_0.52_test_r2")
#X_train, y_train, X_test, y_test, _, _ = data_train_test(train_size=0.7)

INFO:tensorflow:Assets written to: adv_0.52_test_r2/assets


INFO:tensorflow:Assets written to: adv_0.52_test_r2/assets


In [40]:
best_model.predict(adv_X_test)

57/57 [==============================] - 0s 2ms/step


array([[0.5596606 ],
       [0.4603508 ],
       [0.4807027 ],
       ...,
       [0.62426335],
       [0.4381116 ],
       [0.584784  ]], dtype=float32)

In [41]:
adv_y_test

0       0.8
1       0.3
2       0.4
3       0.7
4       0.5
       ... 
1795    0.5
1796    0.9
1797    0.6
1798    0.2
1799    0.6
Name: y, Length: 1800, dtype: float64